# TKET `Backend` tutorial

This example shows how to use `pytket` to execute quantum circuits on both simulators and real devices, and how to interpret the results. As tket is designed to be platform-agnostic, we have unified the interfaces of different providers as much as possible into the `Backend` class for maximum portability of code.

For the full list of supported backends see the pytket [extensions index page](https://tket.quantinuum.com/api-docs/extensions/).

In this notebook we will focus on the Aer, IBMQ and ProjectQ backends.<br>
<br>
To get started, we must install the core pytket package and the subpackages required to interface with the desired providers. We will also need the `QubitOperator` class from `openfermion` to construct operators for a later example. To get everything run the following in shell:<br>
<br>
`pip install pytket pytket-qiskit pytket-projectq openfermion`<br>
<br>
First, import the backends that we will be demonstrating.

In [ ]:
from pytket.extensions.qiskit import (
    AerStateBackend,
    AerBackend,
    AerUnitaryBackend,
)
# from pytket.extensions.projectq import ProjectQBackend

We are also going to be making a circuit to run on these backends, so import the `Circuit` class.

In [ ]:
from pytket import Circuit, Qubit

Below we generate a circuit which will produce a Bell state, assuming the qubits are all initialised in the |0> state:

In [ ]:
circ = Circuit(2)
circ.H(0)
circ.CX(0, 1)

As a sanity check, we will use the `AerStateBackend` to verify that `circ` does actually produce a Bell state.<br>
<br>
To submit a circuit for excution on a backend we can use `process_circuit` with appropriate arguments. If we have multiple circuits to excecute, we can use `process_circuits` (note the plural), which will attempt to batch up the circuits if possible. Both methods return a `ResultHandle` object per submitted `Circuit` which you can use with result retrieval methods to get the result type you want (as long as that result type is supported by the backend).<br>
<br>
Calling `get_state` will return a `numpy` array corresponding to the statevector.<br>
<br>
This style of usage is used consistently in the `pytket` backends.

In [ ]:
aer_state_b = AerStateBackend()
state_handle = aer_state_b.process_circuit(circ)
statevector = aer_state_b.get_result(state_handle).get_state()
print(statevector)

As we can see, the output state vector $\lvert \psi_{\mathrm{circ}}\rangle$ is $(\lvert00\rangle + \lvert11\rangle)/\sqrt2$.<br>
<br>
This is a symmetric state. For non-symmetric states, we default to an ILO-BE format (increasing lexicographic order of (qu)bit ids, big-endian), but an alternative convention can be specified when retrieving results from backends. See the docs for the `BasisOrder` enum for more information.

A lesser-used simulator available through Qiskit Aer is their unitary simulator. This will be somewhat more expensive to run, but returns the full unitary matrix for the provided circuit. This is useful in the design of small subcircuits that will be used multiple times within other larger circuits - statevector simulators will only test that they act correctly on the $\lvert 0 \rangle^{\otimes n}$ state, which is not enough to guarantee the circuit's correctness.<br>
<br>
The `AerUnitaryBackend` provides a convenient access point for this simulator for use with `pytket` circuits. The unitary of the circuit can be retrieved from backends that support it using the `BackendResult.get_unitary` interface. In this example, we chose to use `Backend.run_circuit`, which is equivalent to calling `process_circuit` followed by `get_result`.

In [ ]:
aer_unitary_b = AerUnitaryBackend()
result = aer_unitary_b.run_circuit(circ)
print(result.get_unitary())

Note that state vector and unitary simulations are also available in pytket directly. In general, we recommend you use these unless you require another Backend explicitly.

In [ ]:
statevector = circ.get_statevector()
unitary = circ.get_unitary()

Now suppose we want to measure this Bell state to get some actual results out, so let's append some `Measure` gates to the circuit. The `Circuit` class has the `measure_all` utility function which appends `Measure` gates on every qubit. All of these results will be written to the default classical register ('c'). This function will automatically add the classical bits to the circuit if they are not already there.

In [ ]:
circ.measure_all()

We can get some measured counts out from the `AerBackend`, which is an interface to the Qiskit Aer QASM simulator. Suppose we would like to get 10 shots out (10 repeats of the circuit and measurement). We can seed the simulator's random-number generator in order to make the results reproducible, using an optional keyword argument to `process_circuit`.

In [ ]:
aer_b = AerBackend()
handle = aer_b.process_circuit(circ, n_shots=10, seed=1)

In [ ]:
counts = aer_b.get_result(handle).get_counts()
print(counts)

What happens if we simulate some noise in our imagined device, using the Qiskit Aer noise model?

To investigate this, we will require an import from Qiskit. For more information about noise modelling using Qiskit Aer, see the [Qiskit device noise](https://qiskit.org/documentation/apidoc/aer_noise.html) documentation.

In [ ]:
from qiskit.providers.aer.noise import NoiseModel

In [ ]:
my_noise_model = NoiseModel()
readout_error = 0.2
for q in range(2):
    my_noise_model.add_readout_error(
        [[1 - readout_error, readout_error], [readout_error, 1 - readout_error]], [q]
    )

This simple noise model gives a 20% chance that, upon measurement, a qubit that would otherwise have been measured as $0$ would instead be measured as $1$, and vice versa. Let's see what our shot table looks like with this model:

In [ ]:
noisy_aer_b = AerBackend(my_noise_model)
noisy_handle = noisy_aer_b.process_circuit(circ, n_shots=10, seed=1, valid_check=False)
noisy_counts = noisy_aer_b.get_result(noisy_handle).get_counts()
print(noisy_counts)

We now have some spurious $01$ and $10$ measurements, which could never happen when measuring a Bell state on a noiseless device.<br>
<br>
The `AerBackend` class can accept any Qiskit noise model.<br>
<br>
All backends expose a generic `get_result` method which takes a `ResultHandle` and returns the respective result in the form of a `BackendResult` object. This object may hold measured results in the form of shots or counts, or an exact statevector from simulation. Measured results are stored as `OutcomeArray` objects, which compresses measured bit values into 8-bit integers. We can extract the bitwise values using `to_readouts`.<br>
<br>
Instead of an assumed ILO or DLO convention, we can use this object to request only the `Bit` measurements we want, in the order we want. Let's try reversing the bits of the noisy results.

In [ ]:
backend_result = noisy_aer_b.get_result(noisy_handle)
bits = circ.bits

In [ ]:
outcomes = backend_result.get_counts([bits[1], bits[0]])
print(outcomes)

`BackendResult` objects can be natively serialized to and deserialized from a dictionary. This dictionary can be immediately dumped to `json` for storing results.

In [ ]:
from pytket.backends.backendresult import BackendResult

In [ ]:
result_dict = backend_result.to_dict()
print(result_dict)
print(BackendResult.from_dict(result_dict).get_counts())

The last simulator we will demonstrate is the `ProjectQBackend`. ProjectQ offers fast simulation of quantum circuits with built-in support for fast expectation values from operators. The `ProjectQBackend` exposes this functionality to take in OpenFermion `QubitOperator` instances. These are convertible to and from `QubitPauliOperator` instances in Pytket.<br>
<br>
Note: ProjectQ can also produce statevectors in the style of `AerStateBackend`, and similarly Aer backends can calculate expectation values directly, consult the relevant documentation to see more.<br>
<br>
Let's create an OpenFermion `QubitOperator` object and a new circuit:

In [ ]:
import openfermion as of

In [ ]:
hamiltonian = 0.5 * of.QubitOperator("X0 X2") + 0.3 * of.QubitOperator("Z0")

In [ ]:
circ2 = Circuit(3)
circ2.Y(0)
circ2.H(1)
circ2.Rx(0.3, 2)

We convert the OpenFermion Hamiltonian into a pytket QubitPauliOperator:

In [ ]:
from pytket.pauli import Pauli, QubitPauliString
from pytket.utils.operators import QubitPauliOperator

In [ ]:
pauli_sym = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}

In [ ]:
def qps_from_openfermion(paulis):
    """Convert OpenFermion tensor of Paulis to pytket QubitPauliString."""
    qlist = []
    plist = []
    for q, p in paulis:
        qlist.append(Qubit(q))
        plist.append(pauli_sym[p])
    return QubitPauliString(qlist, plist)

In [ ]:
def qpo_from_openfermion(openf_op):
    """Convert OpenFermion QubitOperator to pytket QubitPauliOperator."""
    tk_op = dict()
    for term, coeff in openf_op.terms.items():
        string = qps_from_openfermion(term)
        tk_op[string] = coeff
    return QubitPauliOperator(tk_op)

In [ ]:
hamiltonian_op = qpo_from_openfermion(hamiltonian)